<a href="https://colab.research.google.com/github/Pmilivojevic/Bioinformatics/blob/main/bioinformatics_chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import copy
from google.colab import drive
drive.mount('/content/drive/')
nucleotides= ['A', 'C', 'G', 'T']

Mounted at /content/drive/


In [ ]:
def reverseComplement(seq):
  revSeq= ''
  for i in range(len(seq)-1, -1, -1):
    if seq[i] == 'A':
      revSeq+= 'T'
    elif seq[i] == 'C':
      revSeq+= 'G'
    elif seq[i] == 'G':
      revSeq+= 'C'
    else:
      revSeq+= 'A'

  return revSeq

In [ ]:
def hammingDistance(p, q):
  d= 0
  for i in range(len(p)):
    if p[i] != q[i]:
      d+= 1

  return d

In [ ]:
print(hammingDistance('CTACAGCAATACGATCATATGCGGATCCGCAGTGGCCGGTAGACACACGT', 'CTACCCCGCTGCTCAATGACCGGGACTAAAGAGGCGAAGATTATGGTGTG'))

36


In [ ]:
def patternCount(seq, pattern):
  count= 0

  for i  in range(len(seq)-len(pattern)+1):
    if seq[i:i+len(pattern)] == pattern:
      count+= 1

  return count

In [ ]:
patternCount('ACTGTACGATGATGTGTGTCAAAG', 'TGT')

4

In [ ]:
def patternMatching(seq, pattern):
  indexes= ''

  for i  in range(len(seq)-len(pattern)+1):
    if seq[i:i+len(pattern)] == pattern:
      indexes= indexes + str(i) + ' '

  return indexes

In [ ]:
def approximatePatternCount(seq, pattern, d):
  count= 0
  ind= []

  for i in range(len(seq)-len(pattern)+1):
    if hammingDistance(seq[i:i+len(pattern)], pattern) <= d:
      count+= 1
      ind.append(i)

  return count
#  return(ind)

In [ ]:
def symbolToNum(c):
  if c == 'A':
    return 0
  elif c == 'C':
    return 1
  elif c == 'G':
    return 2
  else:
    return 3

def patternToNum(pattern):
  if len(pattern) == 0:
    return 0
  else:
    symbol= pattern[len(pattern)-1]
    prefix=  pattern[:len(pattern)-1]
    return 4*patternToNum(prefix) + symbolToNum(symbol)

In [ ]:
def numToSymbol(n):
  if n == 0:
    return 'A'
  elif n == 1:
    return 'C'
  elif n == 2:
    return 'G'
  else:
    return 'T'

def numToPattern(index, k):
  if k == 1:
    return numToSymbol(index)
  else:
    prefixInd= int(index/4)
    remainder= index%4
    symbol= numToSymbol(remainder)
    return numToPattern(prefixInd, k-1) + symbol

In [ ]:
def computingFrequencies(text, k):
  freqArray= np.zeros((4**k))

  for i in range(len(text)-k+1):
    j= patternToNum(text[i:i+k])
    freqArray[j]+= 1

  return freqArray

In [ ]:
def freqWords(seq, k):
  patterns= []
  freqPatterns= []
  count= []

  for i in range(len(seq)-k+1):
    count.append(patternCount(seq, seq[i:i+k]))

  maxCount= max(count)

  for i in range(len(count)):
    if count[i] == maxCount:
      patterns.append(seq[i:i+k])

  freqPatterns.append(patterns[0])

  for i in range(1, len(patterns)):
    for j in range(0, i):
      if patterns[i] == patterns[j]:
        break
    if j == i-1:
      freqPatterns.append(patterns[i])

  return freqPatterns

In [ ]:
print(freqWords('CGGAGGACTCTAGGTAACGCTTATCAGGTCCATAGGACATTCA', 3))

['AGG', 'AGG']


In [ ]:
def fasterFreqWords(seq, k):
  freqPatterns= []
  freqArray= computingFrequencies(seq, k)
  maxFreq= max(freqArray)

  for i in range(len(freqArray)):
    if freqArray[i] == maxFreq:
      freqPatterns.append(numToPattern(i,k))

  return freqPatterns

In [ ]:
def findFreqWordsBySorting(seq, k):
  index= []
  count= []
  paterns= []

  for i in range(len(seq)-k+1):
    index.append(patternToNum(seq[i:i+k]))
    count.append(1)

  index.sort()

  for i in range(1, len(index)):
    if index[i] == index[i-1]:
      count[i]= count[i-1] + 1

  for i in range(len(index)):
    if count[i] == max(count):
      paterns.append(numToPattern(index[i], k))

    return paterns

In [ ]:
def clumpFinding(genom, k, L, t):
  freqPatterns= []
  clumps= np.zeros((4**k))

  for i in range(len(genom)-L+1):
    freqArray= computingFrequencies(genom[i:i+L], k)

    for j in range(len(clumps)):
      if freqArray[j] >= t:
        clumps[j]= 1

  for i in range(len(clumps)):
    if clumps[i] == 1:
      freqPatterns.append(numToPattern(i, k))

  return freqPatterns

In [ ]:
def betterClumpFinding(genom, k, L, t):
  freqPatterns= []
  clumps= np.zeros((4**k))
  freqArray= computingFrequencies(genom[0:L],k)

  for i in range(len(freqArray)):
    if freqArray[i] >= t:
      clumps[i]= 1

  for i in range(len(genom)-L+1):
    index= patternToNum(genom[i-1:i-1+k])
    freqArray[index]= freqArray[index]-1
    index= patternToNum(genom[i+L-k:i+L])
    freqArray[index]= freqArray[index]+1

    if freqArray[index] >= t:
      clumps[index]= 1

  for i in range(len(clumps)):
    if clumps[i] == 1:
      freqPatterns.append(numToPattern(i, k))

  return freqPatterns

In [ ]:
def immediateNeighbors(pattern):
  neighborhood= [pattern]

  for i in range(len(pattern)):
    symbol= pattern[i]

    for X in nucleotides:
      if X != pattern[i]:
        sample= pattern[0:i]+X+pattern[i+1:len(pattern)]
        neighborhood.append(sample)

  return neighborhood

In [ ]:
def sufix(text):
  return text[1:len(text)]

def neighbors(pattern, d):
  if d == 0:
    return pattern

  if len(pattern) == 1:
    return nucleotides

  neighborhood= []
  sufixNeighbors= neighbors(sufix(pattern), d)

  for s in sufixNeighbors:
    if hammingDistance(s, sufix(pattern)) < d:
      for X in nucleotides:
        neighborhood.append(X+s)
    else:
      neighborhood.append(pattern[0]+s)

  return neighborhood

In [ ]:
def freqWordsWithMismatches(seq, k, d):
  freqPatterns= []
  close= np.zeros((4**k))
  freqArray= np.zeros((4**k))

  for i in range(len(seq)-k+1):
    neighborhood= neighbors(seq[i:i+k], d)

    for pattern in neighborhood:
      index= patternToNum(pattern)
      close[index]= 1

  for i in range(len(close)):
    if close[i] == 1:
      pattern= numToPattern(i, k)
      freqArray[i]= approximatePatternCount(seq, pattern, d)

  maxCount= max(freqArray)

  for i in range(len(freqArray)):
    if freqArray[i] == maxCount:
      freqPatterns.append(numToPattern(i,k))

  return freqPatterns

In [ ]:
def freqWordsWithMismatchesBySorting(seq, k, d):
  freqPatterns= []
  neighborhood= []
  index= []
  count= []

  for i in range(len(seq)-k+1):
    neighborhood.append(neighbors(seq[i:i+k], d))

  neighborhoodArray= []

  for i in range(len(neighborhood)):
    for j in range(len(neighborhood[0])):
      neighborhoodArray.append(neighborhood[i][j])

  for i in range(len(neighborhoodArray)):
    index.append(patternToNum(neighborhoodArray[i]))
    count.append(1)

  index.sort()

  for i in range(1,len(index)):
    if index[i] == index[i-1]:
      count[i]= count[i-1]+1

  maxCount= max(count)

  for i in range(len(count)):
    if count[i] == maxCount:
      freqPatterns.append(numToPattern(index[i], k))

  return freqPatterns

In [ ]:
seq= 'AAGCAAAGGTAAGGTGGGGAGCAAAGAAGGTGAGCAAAGAGGTAAGGTGGGGAGCAAAGAAGGTGA'
t= time.time()
print(findFreqWordsBySorting(seq, 2), ' ', time.time()-t)
t= time.time()
print(freqWords(seq, 2), ' ', time.time()-t)
t= time.time()
print(fasterFreqWords(seq, 2), ' ', time.time()-t)
t= time.time()
print(clumpFinding(seq, 2, 9, 2), ' ', time.time()-t)
t= time.time()
print(betterClumpFinding(seq, 2, 9, 2), ' ', time.time()-t)
p= 'AAAAAA'
q= 'TACAAG'
print(hammingDistance(p,q))
print(approximatePatternCount(seq, 'AGAA', 1))
print(immediateNeighbors('AAA'))
print(neighbors('AAA', 1))
t= time.time()
print(freqWordsWithMismatches(seq, 2, 1), ' ', time.time()-t)
t= time.time()
print(freqWordsWithMismatchesBySorting(seq, 2, 1), ' ', time.time()-t)

[]   0.00021076202392578125
['AA', 'AG']   0.0009758472442626953
['AA', 'AG']   0.0003898143768310547
['AA', 'AG', 'GA', 'GG', 'GT']   0.0033113956451416016
['AA', 'AG', 'GA', 'GG', 'GT']   0.00044155120849609375
3
7
['AAA', 'CAA', 'GAA', 'TAA', 'ACA', 'AGA', 'ATA', 'AAC', 'AAG', 'AAT']
['AAA', 'CAA', 'GAA', 'TAA', 'ACA', 'AGA', 'ATA', 'AAC', 'AAG', 'AAT']
['GA']   0.0021719932556152344
['GA']   0.0014507770538330078


In [ ]:
8.7//4

2.0

In [ ]:
def FrequencyMap(Text, k):
    freq = {}
    n = len(Text)
    for i in range(n-k+1):
        Pattern = Text[i:i+k]
        freq[Pattern] = 0
    for k,v in freq.items():
        freq[k]= PatternCount(Text, k)

    return freq

def PatternCount(Text, Pattern):
    count= 0

    for i  in range(len(Text)-len(Pattern)+1):
        if Text[i:i+len(Pattern)] == Pattern:
            count+= 1

    return count

def FrequentWords(Text, k):
    words = []
    freq = FrequencyMap(Text, k)
    m = max(freq.values())
    for key in freq:
        if freq[key] == m:
            words.append(key)
    return words

def Reverse(Pattern):
    revSeq= ''
    for char in Pattern:
        revSeq= char + revSeq

    return revSeq

def Complement(Pattern):
    revSeq= ''
    for char in Pattern:
        if char == 'A':
            revSeq+= 'T'
        elif char == 'C':
            revSeq+= 'G'
        elif char == 'G':
            revSeq+= 'C'
        else:
            revSeq+= 'A'

    return revSeq

def ReverseComplement(Pattern):
    return Reverse(Complement(Pattern))

def PatternMatching(Pattern, Genome):
    positions = []

    for i  in range(len(Genome)-len(Pattern)+1):
        if Genome[i:i+len(Pattern)] == Pattern:
            positions.append(i)

    return positions

def SymbolArray(Genome, symbol):
    array = {}
    n = len(Genome)
    ExtendedGenome = Genome + Genome[0:n//2]
    for i in range(n):
        array[i] = PatternCount(ExtendedGenome[i:i+(n//2)], symbol)
    return array

def FasterSymbolArray(Genome, symbol):
    array = {}
    n = len(Genome)
    ExtendedGenome = Genome + Genome[0:n//2]

    # look at the first half of Genome to compute first array value
    array[0] = PatternCount(symbol, Genome[0:n//2])

    for i in range(1, n):
        # start by setting the current array value equal to the previous array value
        array[i] = array[i-1]

        # the current array value can differ from the previous array value by at most 1
        if ExtendedGenome[i-1] == symbol:
            array[i] = array[i]-1
        if ExtendedGenome[i+(n//2)-1] == symbol:
            array[i] = array[i]+1
    return array

def SkewArray(Genome):
    Skew= np.zeros((len(Genome)+1), dtype= int)

    for i in range(len(Genome)):
        if Genome[i] == 'G':
            Skew[i+1]= Skew[i] + 1
        elif Genome[i] == 'C':
            Skew[i+1]= Skew[i] - 1
        else:
            Skew[i+1]= Skew[i]

    return Skew

def MinimumSkew(Genome):
    positions = [] # output variable

    Skew= SkewArray(Genome)
    for i in range(len(Skew)):
        if Skew[i] == min(Skew):
            positions.append(i)

    return positions

def HammingDistance(p, q):
    d= 0
    for i in range(len(p)):
        if p[i] != q[i]:
            d+= 1

    return d

def ApproximatePatternMatching(Text, Pattern, d):
    positions = []

    for i in range(len(Text)-len(Pattern)+1):
        if HammingDistance(Text[i:i+len(Pattern)], Pattern) <= d:
            positions.append(i)

    return positions

def ApproximatePatternCount(Pattern, Text, d):
    count = 0
    for i in range(len(Text)-len(Pattern)+1):
        if HammingDistance(Text[i:i+len(Pattern)], Pattern) <= d:
            count+= 1

    return count